In [2]:
import numpy as np

**Value Iteration Function:**

In [20]:
# Definition of grid world parameters
rows, cols = 4, 5
obstacles = [(1, 1), (2, 2)]
value_function = np.zeros((rows, cols))

# Grid costs
grid = np.zeros((rows, cols))
# Violet cells (-1)
grid[0, 4] = -1
grid[3, 4] = -1
# Green cells (1)
grid[0, 2] = 1
grid[1, 2] = 1
grid[2, 3] = 1
grid[3, 0] = 1
# Red cells (10)
grid[2, 1] = 10
grid[1, 4] = 10
grid[3, 3] = 10

alpha = 0.99
threshold = 1e-6
actions = [(0,0), (0,1), (0,-1), (1,0), (-1,0)]

def run_value_iteration():
    value_function = np.zeros((rows, cols))
    iterations = 0
    
    while True:
        iterations += 1
        delta = 0
        new_value = np.copy(value_function)
        
        for i in range(rows):
            for j in range(cols):
                if (i,j) in obstacles:
                    continue
                    
                values = []
                for action in actions:
                    next_i = i + action[0]
                    next_j = j + action[1]
                    
                    if (0 <= next_i < rows and 
                        0 <= next_j < cols and 
                        (next_i, next_j) not in obstacles):
                        values.append(grid[i,j] + alpha * value_function[next_i, next_j])
                
                if values:
                    new_value[i,j] = min(values)
                    delta = max(delta, abs(new_value[i,j] - value_function[i,j]))
        
        value_function = new_value
        if delta < threshold:
            break
            
    print(f"\nValue Iteration took {iterations} iterations to converge")
    print("\nFinal Value Function:")
    print(value_function)
    return iterations, value_function

# Run Value Iteration
vi_iterations, vi_value = run_value_iteration()


Value Iteration took 1376 iterations to converge

Final Value Function:
[[-95.07940237 -96.03980137 -97.00990137 -98.99990137 -99.99990137]
 [-94.12860736   0.         -96.02980137 -98.00990137 -88.99990137]
 [-93.1873203  -82.25544611   0.         -97.00990137 -98.99990137]
 [-91.25544611 -90.34289066 -89.43946077 -88.99990137 -99.99990137]]


**Policy Iteration Function**

In [21]:
# Policy Iteration Implementation
def run_policy_iteration():
    # Initialization of the policy to "do nothing" (0,0)
    policy = np.zeros((rows, cols, 2), dtype=int)
    value = np.zeros((rows, cols))
    iterations = 0
    
    while True:
        iterations += 1
        # Policy evaluation
        while True:
            delta = 0
            for i in range(rows):
                for j in range(cols):
                    if (i,j) in obstacles:
                        continue
                        
                    old_value = value[i,j]
                    next_i = i + policy[i,j,0]
                    next_j = j + policy[i,j,1]
                    
                    if (0 <= next_i < rows and 
                        0 <= next_j < cols and 
                        (next_i, next_j) not in obstacles):
                        value[i,j] = grid[i,j] + alpha * value[next_i, next_j]
                    
                    delta = max(delta, abs(value[i,j] - old_value))
            
            if delta < threshold:
                break
                
        # Policy improvement
        policy_stable = True
        for i in range(rows):
            for j in range(cols):
                if (i,j) in obstacles:
                    continue
                    
                old_action = policy[i,j].copy()
                values = []
                
                for action in actions:
                    next_i = i + action[0]
                    next_j = j + action[1]
                    
                    if (0 <= next_i < rows and 
                        0 <= next_j < cols and 
                        (next_i, next_j) not in obstacles):
                        values.append((grid[i,j] + alpha * value[next_i, next_j], action))
                
                if values:
                    best_value, best_action = min(values)
                    policy[i,j] = best_action
                    
                if not np.array_equal(policy[i,j], old_action):
                    policy_stable = False
        
        if policy_stable:
            break
    
    print(f"\nPolicy Iteration took {iterations} iterations to converge")
    print("\nFinal Value Function:")
    print(value)
    return iterations, value, policy

# Run Policy Iteration
pi_iterations, pi_value, pi_policy = run_policy_iteration()


Policy Iteration took 8 iterations to converge

Final Value Function:
[[-95.07949251 -96.03989151 -97.00999151 -98.99999151 -99.99999151]
 [-94.12869758   0.         -96.02989159 -98.00999159 -88.99999159]
 [-93.18741061 -82.2555365    0.         -97.00999151 -98.99999151]
 [-91.2555365  -90.34298114 -89.43955132 -88.99999151 -99.99999151]]
